In [1]:
import os
os.chdir('/content/drive/MyDrive/keep/git/Pretrained-Language-Model/TinyBERT')

In [2]:
import sys, os
lib_path = '/content/drive/MyDrive/keep/git/Pretrained-Language-Model/TinyBERT/libs'

nb_path = '/content/TinyBERT'
# os.symlink(lib_path, nb_path)
sys.path.insert(0,nb_path)

Evaluation

In [16]:
from transformer.modeling import TinyBertForSequenceClassification
from transformer.tokenization import BertTokenizer
import torch

In [15]:
from task_distill import ColaProcessor, MnliProcessor, MnliMismatchedProcessor, MrpcProcessor, Sst2Processor, StsbProcessor,\
    QqpProcessor, QnliProcessor, RteProcessor, WnliProcessor, convert_examples_to_features, get_tensor_data, do_eval

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)


In [25]:
task_name = 'qnli'
data_dir = '/content/drive/MyDrive/keep/datasets/glue_data/QNLI'
model_dir = 'models/teachers-finetuned/qnli'
eval_batch_size = 32

In [26]:
device = torch.device("cuda" if torch.cuda.is_available()  else "cpu")

processors = {
        "cola": ColaProcessor,
        "mnli": MnliProcessor,
        "mnli-mm": MnliMismatchedProcessor,
        "mrpc": MrpcProcessor,
        "sst-2": Sst2Processor,
        "sts-b": StsbProcessor,
        "qqp": QqpProcessor,
        "qnli": QnliProcessor,
        "rte": RteProcessor,
        "wnli": WnliProcessor
    }

output_modes = {
        "cola": "classification",
        "mnli": "classification",
        "mrpc": "classification",
        "sst-2": "classification",
        "sts-b": "regression",
        "qqp": "classification",
        "qnli": "classification",
        "rte": "classification",
        "wnli": "classification"
    }

# intermediate distillation default parameters
default_params = {
        "cola": {"num_train_epochs": 50, "max_seq_length": 64},
        "mnli": {"num_train_epochs": 5, "max_seq_length": 128},
        "mrpc": {"num_train_epochs": 20, "max_seq_length": 128},
        "sst-2": {"num_train_epochs": 10, "max_seq_length": 64},
        "sts-b": {"num_train_epochs": 20, "max_seq_length": 128},
        "qqp": {"num_train_epochs": 5, "max_seq_length": 128},
        "qnli": {"num_train_epochs": 10, "max_seq_length": 128},
        "rte": {"num_train_epochs": 20, "max_seq_length": 128}
    }

processor = processors[task_name]()
output_mode = output_modes[task_name]
label_list = processor.get_labels()
max_seq_length = default_params[task_name]["max_seq_length"]
num_labels = len(label_list)

tokenizer = BertTokenizer.from_pretrained(model_dir)
student_model = TinyBertForSequenceClassification.from_pretrained(model_dir, num_labels=num_labels)
student_model.to(device)

eval_examples = processor.get_dev_examples(data_dir)
eval_features = convert_examples_to_features(eval_examples, label_list, max_seq_length, tokenizer, output_mode)
eval_data, eval_labels = get_tensor_data(output_mode, eval_features)
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

04/28 03:32:01 PM Model config {
  "_name_or_path": "/content/drive/MyDrive/keep/git/Pretrained-Language-Model/TinyBERT/models/bert-base-uncased/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "finetuning_task": "qnli",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "entailment",
    "1": "not_entailment"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "entailment": 0,
    "not_entailment": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "training": "",
  "transformers_version": "4.18.0.dev0",
  "type_vocab_size": 2,
  "

In [27]:
result = do_eval(student_model, task_name, eval_dataloader,
                                         device, output_mode, eval_labels, num_labels)

Evaluating: 100%|██████████| 171/171 [00:23<00:00,  7.28it/s]


In [28]:
result

{'acc': 0.9084751967783269, 'eval_loss': 0.26494505735691526}